# Ratings Prediction

We have a client who has a website where people write different reviews for technical products. Now they are adding a new feature to their website i.e. The reviewer will have to add stars(rating) as well with the review. The rating is out 5 stars and it only has 5 options available 1 star, 2 stars, 3 stars, 4 stars, 5 stars. Now they want to predict ratings for the reviews which were written in the past and they don’t have a rating. So, we have to build an application which can predict the rating by seeing the review.

## Data Collection Phase:

You have to scrape at least 20000 rows of data. You can scrape more data as well, it’s up to you. more the data better the model.

In this section you need to scrape the reviews of different laptops, Phones, Headphones, smart watches, Professional Cameras, Printers, Monitors, Home theater, Router from different ecommerce websites. Basically, we need these columns 1) reviews of the product. 2) rating of the product. You can fetch other data as well, if you think data can be useful or can help in the project. It completely depends on your imagination or assumption.

In [1]:
## importing the libraries:
import selenium
from selenium import webdriver
import pandas as pd
import numpy as np
import time

import re

## importing require exceptions:-
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException,WebDriverException 
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.action_chains import ActionChains

import warnings
warnings.filterwarnings('ignore')

In [2]:
driver=webdriver.Chrome('chromedriver.exe')
driver.maximize_window()
driver

<selenium.webdriver.chrome.webdriver.WebDriver (session="0e78b9b7bf639cd329e69f5e8197c1eb")>

## Scraping from Flipkart.com


In [3]:
## fetching the websites
driver.get("https://www.flipkart.com/")
time.sleep(2)

#clicking on the pop-up Add's cancel button:- 
driver.find_element_by_xpath("//div[@class='_2QfC02']/button").click()

Item_List=['laptops','Smartphones','smart watches' ,'Computer moniters', 
                 'Digital Watches','Cameras','Trimmers','Keyboards']

reviews=[]
full_reviews=[]
ratings=[]

In [4]:
product_urls=[]
for i in Item_List:
    # Find the search bar
    search_button = driver.find_element_by_xpath("//div[@class='_3OO5Xc']/input")
    search_button.clear()
    search_button.send_keys(i)
    #Clicking on the search button
    driver.find_element_by_xpath("//button[@class='L0Z3Pu']").click()
    time.sleep(3)
    
    ## we will scrap atleast 2 pages product of each items
    page_no_url = []
    tag= driver.find_elements_by_xpath("//nav[@class='yFHi8N']/a")
    for i in tag:
        page_no_url.append(i.get_attribute('href'))
    for i in page_no_url[0:3]:
        driver.get(i)
        time.sleep(3)
        
        ## now appending all products urls from page 1 and page 2:
        new_tag = driver.find_elements_by_xpath("//a[@class='_1fQZEK']")
        for i in new_tag:
            product_urls.append(i.get_attribute('href'))
print(len(product_urls))  

288


### Scapping all the reviews of every product:

In [6]:
## iterating all products urls one by one:
for i in product_urls[:]:
    driver.get(i)
    time.sleep(2)
    for _ in range(2):
        ## moving cursor deep down on the page
        driver.execute_script("window.scrollBy(0,6000)")
        time.sleep(1)
        
    # Clicking on "see all the reviews" button:
    try:
        tag=driver.find_element_by_xpath("//div[@class='_2c2kV-']/following::a")
        url = tag.get_attribute('href')
        #fetching the url
        driver.get(url)
        time.sleep(1)
    except NoSuchElementException:
        pass
    try:
        tag=driver.find_elements_by_xpath("//div[@class='t-ZTKy']")
        for i in tag:
            full_reviews.append(i.text)
    except NoSuchElementException:
        full_reviews.append('-')
    try:
        tag=driver.find_elements_by_xpath("//p[@class='_2-N8zT']")
        for i in tag:
            reviews.append(i.text)
    except NoSuchElementException:
        reviews.append('-')
    try:
        tag=driver.find_elements_by_xpath("//p[@class='_2-N8zT']/../div")
        for i in tag:
            ratings.append(i.text)
    except NoSuchElementException:
        ratings.append('-')
               
    try:
        no_of_pages=driver.find_elements_by_xpath("//nav[@class='yFHi8N']/a")
        ListOfNP=[]
        for i in no_of_page:
            ListOfNP.append(i.get_attribute('href'))
        for i in ListOfNP[0:15]:
            driver.get(i)
            time.sleep(1)
            try:
                tag=driver.find_elements_by_xpath("//div[@class='t-ZTKy']")
                for i in tag:
                    full_reviews.append(i.text)
            except NoSuchElementException:
                full_reviews.append('-')
            try:
                tag=driver.find_elements_by_xpath("//p[@class='_2-N8zT']")
                for i in tag:
                    reviews.append(i.text)
            except NoSuchElementException:
                reviews.append('-')
            try:
                tag=driver.find_elements_by_xpath("//p[@class='_2-N8zT']/../div")
                for i in tag:
                    ratings.append(i.text)
            except NoSuchElementException:
                ratings.append('-')    
    except: continue
len(ratings), len(reviews), len(full_reviews)  

(3611, 3611, 3611)

In [7]:
#Creating a dataframe
df=pd.DataFrame()
df['Ratings']=ratings
df['Review_Text']=full_reviews
df['Summary']=reviews
df

,Ratings,Review_Text,Summary
0,5,"Amazing product in price range , good sound qu...",Fabulous!
1,3,Good,Nice
2,5,Thank you flipkart for thiz amazing product..d...,Simply awesome
3,5,Good,Terrific purchase
4,5,Awesome Product.,Simply awesome
...,...,...,...
3606,5,Awesome deal got.,Perfect product!
3607,1,Worst. Don’t ever buy this,Don't waste your money
3608,5,Value for money product.\n\nDisplay quality: 5...,Must buy!
3609,5,"Excellent Colour, Excellent Display,Worth Ever...",Simply awesome


In [8]:
df.to_csv("Flipkart_Ratings.csv")

In [9]:
driver.close()

## Scraping from Amazon.in

In [10]:
driver=webdriver.Chrome('chromedriver.exe')
driver.maximize_window()
driver

<selenium.webdriver.chrome.webdriver.WebDriver (session="c46ff9a16717d24814aa829ec2c7617d")>

In [11]:
### creating a list of Items, for which we have to scrap their ratings
List_of_items = ['laptops','Smartphones','smart watches' , 'Printers','Computer moniters', 
                 'Digital Watches', 'Router','Cameras','Trimmers','Keyboards']

In [12]:
title = []
review_text = []
ratings = []

In [14]:
## Now we will create a empty list and add all the  item's URLs present in page one.
Product_URL=[]

## Iterating items one by one in the loop and storing the urls
for i in List_of_items:
    ## fetching  the website
    driver.get("https://www.amazon.in/")
    time.sleep(3)
    
    srchbar = driver.find_element_by_id("twotabsearchtextbox")
    srchbar.send_keys(i)
    
    #clicking on search button
    
    driver.find_element_by_id("nav-search-submit-button").click()  
    time.sleep(1)
   
  
    tag=driver.find_elements_by_xpath("//h2[@class='a-size-mini a-spacing-none a-color-base s-line-clamp-2']/a")
    for i in tag:
        Product_URL.append(i.get_attribute('href'))

In [15]:
## Priting the length of the list which contains all the product urls
print(len(Product_URL))

276


In [22]:
## Now we create five New empty list for every type of Star rating button's url.
Five_star_rating_URL=[]
Four_star_rating_URL=[]
Three_star_rating_URL=[]
Two_star_rating_URL=[]
One_star_rating_URL=[]

### Iterating all the product URL
for i in Product_URL[:250]:
    driver.get(i)
    time.sleep(1)
    for _ in range(2):
        ## scolling the cursor at down of the page
        driver.execute_script("window.scrollBy(0,6000)")
        time.sleep(1)
        # Clicking on "see all reviews" Button
        try:
            driver.find_element_by_xpath("//a[@class='a-link-emphasis a-text-bold']").click()
        except:
            pass
        
        try:
            tag1= driver.find_element_by_xpath("//table[@id='histogramTable']/tbody/tr[5]/td[2]/a")
            One_star_rating_URL.append(tag1.get_attribute('href'))
        except:
            pass
        try:
            tag2 = driver.find_element_by_xpath("//table[@id='histogramTable']/tbody/tr[4]/td[2]/a")
            Two_star_rating_URL.append(tag2.get_attribute('href'))
        except:
            pass
        try:
            tag3 = driver.find_element_by_xpath("//table[@id='histogramTable']/tbody/tr[3]/td[2]/a")
            for i in tag3:
                Three_star_rating_URL.append(tag3.get_attribute('href'))
        except:
            pass
        try:
            tag4 = driver.find_element_by_xpath("//table[@id='histogramTable']/tbody/tr[2]/td[2]/a")
            Four_star_rating_URL.append(tag4.get_attribute('href'))
        except:
            pass
        try:
            tag5 = driver.find_element_by_xpath("//table[@id='histogramTable']/tbody/tr[1]/td[2]/a")
            Five_star_rating_URL.append(tag5.get_attribute('href'))
        except:
            pass       

In [23]:
print(len(One_star_rating_URL))
print(len(Two_star_rating_URL))
print(len(Three_star_rating_URL))
print(len(Four_star_rating_URL))
print(len(Five_star_rating_URL))

442
424
0
450
464


### "1" Star Ratings reviews


In [24]:
## As we know one_star_rating list contains only 1Star rating reviews so,
reviews_1=[]
Full_reviews_1=[]
for i in One_star_rating_URL:
    driver.get(i)
    time.sleep(1)
    
   
    try:
        tag=driver.find_elements_by_xpath('//div[@class="a-section a-spacing-none review-views celwidget"]/div/div/div/div/a[@data-hook="review-title"]')
        for i in tag:
            reviews_1.append(i.text)
    except NoSuchElementException:
        reviews_1.append('-')
    
    try:
        tag=driver.find_elements_by_xpath('//div[@class="a-section a-spacing-none review-views celwidget"]/div/div/div/div[@class="a-row a-spacing-small review-data"]')
        for i in tag:
            Full_reviews_1.append(i.text)
    except NoSuchElementException:
        Full_reviews_1.append('-')  

In [25]:
print(len(reviews_1),len(Full_reviews_1))

3672 3756


In [26]:
## creating a list of Ratings:
ratings_1=[1]*3672

### "2" Star Rating Reviews:

In [27]:
## As we know two_star_rating list contains only 2 Star rating reviews so,
reviews_2=[]
Full_reviews_2=[]

for i in Two_star_rating_URL[:300]:
    driver.get(i)
    time.sleep(1)
    try:
        tag=driver.find_elements_by_xpath('//div[@class="a-section a-spacing-none review-views celwidget"]/div/div/div/div/a[@data-hook="review-title"]')
        for i in tag:
            reviews_2.append(i.text)
    except NoSuchElementException:
        reviews_2.append('-')
    
    try:
        tag=driver.find_elements_by_xpath('//div[@class="a-section a-spacing-none review-views celwidget"]/div/div/div/div[@class="a-row a-spacing-small review-data"]')
        for i in tag:
            Full_reviews_2.append(i.text)
    except NoSuchElementException:
        Full_reviews_2.append('-') 

In [28]:
print(len(reviews_2),len(Full_reviews_2))

2206 2268


In [29]:
### creating 2 Star rating list:
ratings_2=[2]*2206

### "3" Star Rating Reviews:-

In [30]:
## As we know three_star_rating list contains only 3 Star rating reviews so,
reviews_3=[]
Full_reviews_3=[]

for i in Three_star_rating_URL:
    driver.get(i)
    time.sleep(1)
   
    try:
        tag=driver.find_elements_by_xpath('//a[@data-hook="review-title"]')
        for i in tag:
            reviews_3.append(i.text)
    except NoSuchElementException:
        reviews_3.append('-')
    
    try:
        tag=driver.find_elements_by_xpath('//div[@class="a-row a-spacing-small review-data"]//span[@data-hook="review-body"]')
        for i in tag:
            Full_reviews_3.append(i.text)
    except NoSuchElementException:
        Full_reviews_3.append('-') 

In [31]:
print(len(reviews_3),len(Full_reviews_3))

0 0


In [32]:
## Creating a 3 Star rating list
ratings_3=[3]*0

### "4" Star Rating Reviews

In [35]:
## As we know Four_star_rating list contains only 4 Star rating reviews so,
reviews_4=[]
Full_reviews_4=[]

for i in Four_star_rating_URL:
    driver.get(i)
    time.sleep(1)
   
    try:
        tag=driver.find_elements_by_xpath('//div[@class="a-section a-spacing-none review-views celwidget"]/div/div/div/div/a[@data-hook="review-title"]')
        for i in tag:
            reviews_4.append(i.text)
    except NoSuchElementException:
        reviews_4.append('-')
    
    try:
        tag=driver.find_elements_by_xpath('//div[@class="a-section a-spacing-none review-views celwidget"]/div/div/div/div[@class="a-row a-spacing-small review-data"]')
        for i in tag:
            Full_reviews_4.append(i.text)
    except NoSuchElementException:
        Full_reviews_4.append('-')  

In [36]:
print(len(reviews_4),len(Full_reviews_4))

3616 3732


In [37]:
### Creating a list of 4 Star Reviews:
ratings_4=[4]*3616

### "5" Star Rating Reviews:

In [38]:
## As we know Five_star_rating list contains only 5 Star rating reviews so,
reviews_5=[]
Full_reviews_5=[]

for i in Five_star_rating_URL:
    driver.get(i)
    time.sleep(1)
   
    try:
        tag=driver.find_elements_by_xpath('//a[@data-hook="review-title"]')
        for i in tag:
            reviews_5.append(i.text)
    except NoSuchElementException:
        reviews_5.append('-')
    
    try:
        tag=driver.find_elements_by_xpath('//div[@class="a-row a-spacing-small review-data"]//span[@data-hook="review-body"]')
        for i in tag:
            Full_reviews_5.append(i.text)
    except NoSuchElementException:
        Full_reviews_5.append('-')  

In [39]:
print(len(reviews_5),len(Full_reviews_5))

4092 4218


In [40]:
## Creating a 5 Star ratings reviews:
ratings_5=[5]*4092

### Adding All the Reviews and Ratings:


In [41]:
Reviews = reviews_1 + reviews_2 + reviews_3 + reviews_4 + reviews_5
Full_Reviews = Full_reviews_1[:3672] + Full_reviews_2[0:2206] + Full_reviews_3 + Full_reviews_4[:3616] + Full_reviews_5[:4092]
Ratings = ratings_1 + ratings_2 + ratings_3 + ratings_4 + ratings_5

### Creating A DataFrame:


In [42]:
new_df=pd.DataFrame()
new_df['Ratings']=Ratings
new_df['Review_Text']=Full_Reviews
new_df['Summary']=Reviews
new_df

,Ratings,Review_Text,Summary
0,1,Amazon is selling used product as new product....,Not worth the money.
1,1,Please don’t buy any electronic devices in Ama...,Worst experience with Amazon
2,1,"Hi, Amazon told that we get 12 months Warrenty...",Not original product. Loads of issues. Do not buy
3,1,Product has some warrantee related problem. Re...,Product not good
4,1,Amazon customer service is bad as they are not...,Speaker sound is cracking while video playback.
...,...,...,...
13581,5,The camera is really budjet friendly\nThe qual...,Amazon delivered this product just in one day ...
13582,5,"Price, video quality is best in budget company...",Nice product
13583,5,Value for money product. Very good stabilizati...,Smooth ... Feel standard
13584,5,Play Video\n Pro's:\n1. Good clarity consideri...,Osm


In [44]:
new_df.to_csv("Amazon_Ratings.csv")

### Adding All Data


In [45]:
data=pd.concat([df,new_df],axis=0)
data

,Ratings,Review_Text,Summary
0,5,"Amazing product in price range , good sound qu...",Fabulous!
1,3,Good,Nice
2,5,Thank you flipkart for thiz amazing product..d...,Simply awesome
3,5,Good,Terrific purchase
4,5,Awesome Product.,Simply awesome
...,...,...,...
13581,5,The camera is really budjet friendly\nThe qual...,Amazon delivered this product just in one day ...
13582,5,"Price, video quality is best in budget company...",Nice product
13583,5,Value for money product. Very good stabilizati...,Smooth ... Feel standard
13584,5,Play Video\n Pro's:\n1. Good clarity consideri...,Osm


In [46]:
#saving the data into excel file
data.to_excel("Ratings_Dataset.xlsx")

In [47]:
driver.close()